<a href="https://colab.research.google.com/github/yananma/5_programs_per_day/blob/master/02114.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 10.6 求近义词和类比词

### 10.6.1 使用预训练的词向量

In [3]:
import torch 
import torchtext.vocab as vocab 

vocab.pretrained_aliases.keys()

dict_keys(['charngram.100d', 'fasttext.en.300d', 'fasttext.simple.300d', 'glove.42B.300d', 'glove.840B.300d', 'glove.twitter.27B.25d', 'glove.twitter.27B.50d', 'glove.twitter.27B.100d', 'glove.twitter.27B.200d', 'glove.6B.50d', 'glove.6B.100d', 'glove.6B.200d', 'glove.6B.300d'])

In [4]:
[key for key in vocab.pretrained_aliases.keys() if 'glove' in key]

['glove.42B.300d',
 'glove.840B.300d',
 'glove.twitter.27B.25d',
 'glove.twitter.27B.50d',
 'glove.twitter.27B.100d',
 'glove.twitter.27B.200d',
 'glove.6B.50d',
 'glove.6B.100d',
 'glove.6B.200d',
 'glove.6B.300d']

In [5]:
glove = vocab.GloVe(name='6B', dim=50)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                          
100%|█████████▉| 398476/400000 [00:09<00:00, 43303.23it/s]


In [6]:
len(glove.stoi)

400000

In [7]:
glove.stoi['beautiful'], glove.itos[3366]

(3366, 'beautiful')

### 10.6.2 应用预训练词向量

#### 1 求近义词

In [0]:
def knn(W, x, k):
    cos = torch.matmul(W, x.view((-1, ))) / (
        (torch.sum(W * W, dim=1) + 1e-9).sqrt() * torch.sum(x * x).sqrt())
    _, topk = torch.topk(cos, k=k)
    topk = topk.cpu().numpy()
    return topk, [cos[i].item() for i in topk]

In [0]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.vectors, 
             embed.vectors[embed.stoi[query_token]], k+1)
    for i, c in zip(topk[1:], cos[1:]):
        print('cosine sim=%.3f: %s' % (c, (embed.itos[i])))

In [13]:
get_similar_tokens('chip', 3, glove)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics


In [14]:
get_similar_tokens('baby', 3, glove)

cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl


In [15]:
get_similar_tokens('beautiful', 3, glove)

cosine sim=0.921: lovely
cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful


#### 2 求类比词

In [0]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = [embed.vectors[embed.stoi[t]] for t in [token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.vectors, x, 1)
    return embed.itos[topk[0]]

In [18]:
get_analogy('man', 'woman', 'son', glove)

'daughter'

In [19]:
get_analogy('beijing', 'china', 'tokyo', glove)

'japan'

In [20]:
get_analogy('bad', 'worst', 'big', glove)

'biggest'

In [21]:
get_analogy('do', 'did', 'go', glove)

'went'